In [6]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import pandas.util.testing as tm
import urllib.request 
import statsmodels.api as sm
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets, linear_model
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split


In [18]:
df = pd.read_csv("output_coffee.csv")
print(df.shape)
df = df.loc[df['Followers'] < 1000000]
print(df.shape)
df = df.loc[df['Likes'] > 0]
print(df.shape)
df["likes_normalized"] = df["Likes"]/df["Likes"].max() 
df["comments_normalized"] = df["Comments"]/df["Comments"].max()
df["engagement_score"] = 0.5*df["likes_normalized"] + 0.5*df["comments_normalized"]

(1997, 14)
(1579, 14)
(1579, 14)


In [9]:
df["no_of_hashtags"] = df["Tags"].str.split(' ').str.len()

In [10]:
df.shape

(1579, 18)

In [11]:
import datetime as dt
import time
df['Created on'] = pd.to_datetime(df['Created at'])
#print(df['Created on'].dt.dayofweek)
df['Day of week'] = df['Created on'].dt.dayofweek
df['Created on'] = df['Created on'].apply(lambda  var: time.mktime(var.timetuple()))

In [6]:
df.head()

,Network,ID,Created at,Author name,Author handle,Followers,Likes,Comments,Tags,Text,URL,image,technical_score,aesthetic_score,likes_normalized,comments_normalized,engagement_score,no_of_hashtags,Created on,Day of week
0,Instagram,instagram_graph_media:CRq2E0xpve7,"Jul 23, 2021 5:49 PM",ryguy8989,ryguy8989,170549.0,29883,352,#bekind #choices #coffee #dobetter #goodmornin...,"""Do the best you can, until you know better. O...",http://instagr.am/p/CRq2E0xpve7/,https://scontent.cdninstagram.com/v/t51.2885-1...,5.362762,5.337581,0.371356,0.209524,0.290440,18,1.627063e+09,4
1,Instagram,instagram_graph_media:CRkQaA6jYkk,"Jul 21, 2021 4:25 AM",therisingsuncoffee,therisingsuncoffee,181597.0,19836,80,#TRSC #coffee #coffee_inst #coffeeaddict #coff...,◆GOOD MORNING COFFEE‼︎◆【INFORMATION】TRSC大網店、グラ...,http://instagr.am/p/CRkQaA6jYkk/,https://scontent.cdninstagram.com/v/t51.2885-1...,5.251076,4.846176,0.246502,0.047619,0.147060,23,1.626842e+09,2
2,Instagram,instagram_graph_media:CRgtYKLFsia,"Jul 19, 2021 7:21 PM",istanbul_gallery_mashhad,istanbul_gallery_mashhad,539266.0,16682,475,#cofee_bar #coffee #espersso #اسمگ #اسمگ_اسپرو...,الوعده وفا...🤩به امید خدا از امروز سفارشگیری ا...,http://instagr.am/p/CRgtYKLFsia/,https://scontent.cdninstagram.com/v/t51.2885-1...,6.373529,5.460973,0.207307,0.282738,0.245023,20,1.626722e+09,0
3,Instagram,instagram_graph_media:CRjnzjZj7YO,"Jul 20, 2021 10:30 PM",topikmagazine,topikmagazine,46575.0,10682,100,#americano #cafe #coffee #culturakpop #kdrama ...,"""Varios de nosotros recurrimos a ciertas bebid...",http://instagr.am/p/CRjnzjZj7YO/,https://scontent.cdninstagram.com/v/t51.2885-1...,5.047775,5.809383,0.132745,0.059524,0.096134,21,1.626820e+09,1
4,Instagram,instagram_graph_media:CRgnZsZqxM5,"Jul 19, 2021 6:29 PM",schwarzer.kaffee,schwarzer.kaffee,415193.0,9100,87,#Kaffee #coffee #gedanken #goodmorning #liebe ...,#Kaffee #coffee #goodmorning #witzig #spaß #sc...,http://instagr.am/p/CRgnZsZqxM5/,https://scontent.cdninstagram.com/v/t51.2885-1...,4.883592,5.772449,0.113086,0.051786,0.082436,11,1.626719e+09,0


In [12]:
scaler = MinMaxScaler()
num_vars = ['Followers','technical_score','aesthetic_score','no_of_hashtags', 'Day of week']
df[num_vars] = scaler.fit_transform(df[num_vars])
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [8]:
df.shape

(1579, 20)

In [15]:
Y = df['engagement_score']
X = df[num_vars]
np.random.seed(0)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.75, test_size = 0.25, random_state = 50)
reg = linear_model.LinearRegression()
reg.fit(X_train, Y_train)
print('Coefficients: ', reg.coef_)

Coefficients:  [ 0.08492894  0.05573187 -0.05246975  0.06378348  0.0087439 ]


In [16]:
print(reg.score(X_test, Y_test))
print(X_train.head())

0.27208959180405057
      Followers  technical_score  aesthetic_score  no_of_hashtags  Day of week
22     0.031065         0.261831         0.642972        0.586207     0.000000
1117   0.729335         0.510730         0.813640        0.965517     0.333333
1060   0.089142         0.714380         0.594671        0.965517     0.166667
1852   0.729335         0.222113         0.489543        0.965517     1.000000
12     0.857518         0.571146         0.421217        0.068966     0.333333


In [17]:
mse_sum = 0
for i in range(len(X_test)):
    prediction = reg.predict([X_test.iloc[i,]])
    mse_sum += (Y_test.iloc[i] - prediction)**2
mse = mse_sum / len(X_test)

print(mse)

[0.00331313]


In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(50, input_dim=5, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

In [13]:
# Compile model
model.compile(loss='mape', optimizer='adam', metrics=['mse'])

# Fit the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test,Y_test))

NameError: name 'model' is not defined

In [14]:
mape, mse = model.evaluate(X_test, Y_test, verbose = 0)
print(mape, mse)

46.66354751586914 0.003953929524868727
